imports

In [ ]:
import numpy as np
from collections import deque
from itertools import combinations_with_replacement, product
from random import random, randrange, shuffle
from scipy.special import binom

utils.py

In [ ]:
def index_in_bounds(i, grid_size):
    return -1 < i < grid_size


def get_neighbors(pair, grid_size):
    x, y = pair
    
    possible_neighbors = [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]
    
    neighbors = [
        (x_neighbor, y_neighbor)
        for x_neighbor, y_neighbor in possible_neighbors
        if (index_in_bounds(x_neighbor, grid_size) and
            index_in_bounds(y_neighbor, grid_size))
    ]
    
    return neighbors


def get_open_neighbors(pair, spaceship):
    neighbors = get_neighbors(pair, spaceship.grid_size)
    
    open_neighbors = [
        (x_neighbor, y_neighbor)
        for x_neighbor, y_neighbor in neighbors
        if spaceship.grid[x_neighbor][y_neighbor]
    ]
    
    return open_neighbors


def get_closed_neighbors(pair, spaceship):
    neighbors = get_neighbors(pair, spaceship.grid_size)
    
    closed_neighbors = [
        (x_neighbor, y_neighbor)
        for x_neighbor, y_neighbor in neighbors
        if not spaceship.grid[x_neighbor][y_neighbor]
    ]
    
    return closed_neighbors


def get_num_open_neighbors(pair, spaceship):
    return len(get_open_neighbors(pair, spaceship))

search.py

In [ ]:
def breadth_first_search(spaceship, source):
    queue = deque()
    dist_source = {}
    
    queue.append(source)
    dist_source[source] = 0
    while queue:
        pair = queue.popleft()
        neighbors = get_open_neighbors(pair, spaceship)
        for neighbor in neighbors:
            if neighbor not in dist_source:
                queue.append(neighbor)
                dist_source[neighbor] = dist_source[pair] + 1
    
    return dist_source

actions.py

In [ ]:
def place_bot(spaceship):
    bot = None
    while not bot:
        x = randrange(0, spaceship.grid_size)
        y = randrange(0, spaceship.grid_size)
        if spaceship.grid[x][y]:
            bot = (x, y)
    return bot


def place_crew_members(spaceship, bot, num_crew_members):
    crew_members = []
    while len(crew_members) < num_crew_members:
        x = randrange(0, spaceship.grid_size)
        y = randrange(0, spaceship.grid_size)
        new_crew_member = (x, y)
        if (spaceship.grid[x][y] and
            new_crew_member not in crew_members and
            new_crew_member != bot):
            crew_members.append(new_crew_member)
    return crew_members


def place_aliens(spaceship, bot, num_aliens, apothem):
    x_bot, y_bot = bot
    aliens = []
    while len(aliens) < num_aliens:
        x = randrange(0, spaceship.grid_size)
        y = randrange(0, spaceship.grid_size)
        new_alien = (x, y)
        if ((abs(x - x_bot) > apothem or abs(y - y_bot) > apothem) and
            spaceship.grid[x][y] and
            # new_alien not in aliens and  # if aliens must be in different cells
            new_alien != bot):
            aliens.append(new_alien)
    return aliens


def place_all(spaceship, num_crew_members, num_aliens, apothem):
    bot = place_bot(spaceship)
    crew_members = place_crew_members(spaceship, bot, num_crew_members)
    aliens = place_aliens(spaceship, bot, num_aliens, apothem)
    return bot, crew_members, aliens


def move_aliens(spaceship, aliens):
    num_aliens = len(aliens)
    
    shuffle(aliens)
    
    for i in range(0, num_aliens):
        next = get_open_neighbors(aliens[i], spaceship)
        # next.append(aliens[i])  # if alien can stay in place
        rand_num = randrange(0, len(next))
        aliens[i] = next[rand_num]

ship.py

In [ ]:
class Spaceship:
    def __init__(self, grid_size, num_aliens = 1):
        self.grid_size = grid_size
        
        # True if open, False if closed
        self.grid = np.full((self.grid_size, self.grid_size), False)
        
        x = randrange(1, self.grid_size - 1)
        y = randrange(1, self.grid_size - 1)
        self.grid[x][y] = True
        
        closed_cells = self.get_closed_cells_with_one_open_neighbor()
        while closed_cells:
            rand_num = randrange(0, len(closed_cells))
            x, y = closed_cells[rand_num]
            self.grid[x][y] = True
            closed_cells = self.get_closed_cells_with_one_open_neighbor()
        
        open_cells = self.get_open_cells_with_one_open_neighbor()
        for pair in open_cells:
            if randrange(0, 2):
                self.open_one_closed_neighbor(pair)
        
        self.num_open_cells = np.count_nonzero(self.grid)
        
        self.grid_neighbors = [
            [[] for _ in range(self.grid_size)] for _ in range(self.grid_size)
        ]
        self.grid_prob_alien_move_neighbor = (
            np.zeros((self.grid_size, self.grid_size))
        )
        self.init_neighbor_arrays()
        
        self.pairs_neighboring_pairs = None
        if num_aliens == 2:
            self.pairs_neighboring_pairs = self.init_pairs_neighboring_pairs()
        
        self.distances = self.init_distances()
    
    def get_closed_cells_with_one_open_neighbor(self):
        closed_cells = []
        
        for i in range(0, self.grid_size):
            for j in range(0, self.grid_size):
                if not self.grid[i][j]:
                    if get_num_open_neighbors((i, j), self) == 1:
                        closed_cells.append((i, j))
        
        return closed_cells
    
    def get_open_cells_with_one_open_neighbor(self):
        open_cells = []
        
        for i in range(0, self.grid_size):
            for j in range(0, self.grid_size):
                if self.grid[i][j]:
                    if get_num_open_neighbors((i, j), self) == 1:
                        open_cells.append((i, j))
        
        return open_cells
    
    def open_one_closed_neighbor(self, pair):
        closed_neighbors = get_closed_neighbors(pair, self)
        
        if closed_neighbors:
            rand_num = randrange(0, len(closed_neighbors))
            x_neighbor, y_neighbor = closed_neighbors[rand_num]
            self.grid[x_neighbor][y_neighbor] = True
    
    def init_neighbor_arrays(self):
        for i in range(0, self.grid_size):
            for j in range(0, self.grid_size):
                if self.grid[i][j]:
                    open_neighbors = get_open_neighbors((i, j), self)
                    self.grid_neighbors[i][j] = open_neighbors
                    self.grid_prob_alien_move_neighbor[i][j] = (
                        1 / len(open_neighbors)
                    )
    
    def init_pairs_neighboring_pairs(self):
        # get open cells
        open_cells = np.argwhere(self.grid).tolist()  # list of lists
        
        # get pairs of open cells
        pairs = (
            list(combinations_with_replacement(open_cells, 2))
        )  # list of tuples of lists
        
        # get neighboring pairs of pairs of open cells
        pairs_neighboring_pairs = {}
        for cell_1, cell_2 in pairs:
            x1, y1 = cell_1
            x2, y2 = cell_2
            neighbors_1 = self.grid_neighbors[x1][y1]  # list of tuples
            neighbors_2 = self.grid_neighbors[x2][y2]  # list of tuples
            pairs_neighboring_pairs[((x1, y1), (x2, y2))] = (
                list(combinations_with_replacement(neighbors_1, 2))
                if cell_1 == cell_2
                else
                list(product(neighbors_1, neighbors_2))
            )  # list of tuples of tuples
        
        return pairs_neighboring_pairs
    
    def init_distances(self):
        distances = {}
        for i in range(0, self.grid_size):
            for j in range(0, self.grid_size):
                if self.grid[i][j]:
                    distances[(i, j)] = breadth_first_search(self, (i, j))
        return distances

bots.py

In [ ]:
def get_detection_square_slices(spaceship, bot, apothem):
    x_bot, y_bot = bot
    
    up = max(x_bot - apothem, 0)
    down = min(x_bot + apothem, spaceship.grid_size - 1)
    left = max(y_bot - apothem, 0)
    right = min(y_bot + apothem, spaceship.grid_size - 1)
    
    slice_x = slice(up, down + 1)
    slice_y = slice(left, right + 1)
    
    return slice_x, slice_y


def get_pairs_with_replacement(cond):
    return np.multiply.outer(cond, cond)


def condition_inside_detection_square(spaceship, bot, apothem):
    slice_x, slice_y = get_detection_square_slices(spaceship, bot, apothem)
    
    cond = np.full((spaceship.grid_size, spaceship.grid_size), False)
    cond[slice_x, slice_y] = spaceship.grid[slice_x, slice_y]
    
    return cond


def condition_outside_detection_square(spaceship, bot, apothem):
    slice_x, slice_y = get_detection_square_slices(spaceship, bot, apothem)
    
    cond = spaceship.grid.copy()
    cond[slice_x, slice_y] = False
    
    return cond


def condition_pairs_without_replacement(spaceship, bot):
    grid_size = spaceship.grid_size
    x_bot, y_bot = bot
    
    # pairs without replacement
    cond = np.outer(spaceship.grid.reshape(-1), spaceship.grid.reshape(-1))
    np.fill_diagonal(cond, False)  # no replacement
    cond.shape = (grid_size, grid_size, grid_size, grid_size)
    
    # exclude pairs with bot
    cond[x_bot, y_bot, :, :] = False
    cond[:, :, x_bot, y_bot] = False
    
    return cond


def condition_pairs_with_replacement(spaceship, bot):
    x_bot, y_bot = bot
    
    # pairs with replacement
    cond = get_pairs_with_replacement(spaceship.grid)
    
    # exclude pairs with bot
    cond[x_bot, y_bot, :, :] = False
    cond[:, :, x_bot, y_bot] = False
    
    return cond


def condition_pairs_with_replacement_inside_detection_square(
    spaceship, bot, apothem
):
    cond_pairs = condition_pairs_with_replacement(spaceship, bot)
    cond_pairs_outside = (
        condition_pairs_with_replacement_outside_detection_square(
            spaceship, bot, apothem
        )
    )
    cond_pairs_inside = np.logical_xor(cond_pairs, cond_pairs_outside)
    return cond_pairs_inside


def condition_pairs_with_replacement_outside_detection_square(
    spaceship, bot, apothem
):
    cond_outside = condition_outside_detection_square(spaceship, bot, apothem)
    cond_pairs_outside = get_pairs_with_replacement(cond_outside)
    return cond_pairs_outside


def init_prob_alien_1(spaceship, bot, apothem):
    cond_outside = condition_outside_detection_square(spaceship, bot, apothem)
    
    p = 1 / np.count_nonzero(cond_outside)
    
    prob_alien_1 = np.where(cond_outside, p, 0)
    
    return prob_alien_1


def update_prob_alien_1_not_found(prob_alien_1, bot):
    x_bot, y_bot = bot
    
    p_alien_in_bot = prob_alien_1[x_bot, y_bot]
    p_alien_not_in_bot = 1 - p_alien_in_bot
    
    prob_alien_1[x_bot, y_bot] = 0
    prob_alien_1 /= p_alien_not_in_bot


def update_prob_alien_1_detected(prob_alien_1, spaceship, bot, apothem):
    cond_outside = condition_outside_detection_square(spaceship, bot, apothem)
    prob_alien_1[cond_outside] = 0
    
    """
    added if statement to fix error when running Bot 6
        RuntimeWarning: invalid value encountered in divide
        prob_alien_1 /= sum
    """
    if np.any(prob_alien_1):
        sum = np.sum(prob_alien_1)
        prob_alien_1 /= sum
    else:
        x_bot, y_bot = bot
        
        cond_inside = (
            condition_inside_detection_square(spaceship, bot, apothem)
        )
        
        p = 1 / (np.count_nonzero(cond_inside) - 1)
        
        prob_alien_1[cond_inside] = p
        prob_alien_1[x_bot, y_bot] = 0


def update_prob_alien_1_not_detected(prob_alien_1, spaceship, bot, apothem):
    cond_inside = condition_inside_detection_square(spaceship, bot, apothem)
    prob_alien_1[cond_inside] = 0
    
    """
    added if statement to fix error when running Bot 6
        RuntimeWarning: invalid value encountered in divide
        prob_alien_1 /= sum
    """
    if np.any(prob_alien_1):
        sum = np.sum(prob_alien_1)
        prob_alien_1 /= sum
    else:
        cond_outside = (
            condition_outside_detection_square(spaceship, bot, apothem)
        )
        p = 1 / np.count_nonzero(cond_outside)
        prob_alien_1[cond_outside] = p


def update_prob_alien_1_alien_moves(prob_alien_1, spaceship):
    prob = prob_alien_1 * spaceship.grid_prob_alien_move_neighbor
    
    for i in range(0, spaceship.grid_size):
        for j in range(0, spaceship.grid_size):
            if spaceship.grid[i][j]:
                p = 0
                for x_neighbor, y_neighbor in spaceship.grid_neighbors[i][j]:
                    p += prob[x_neighbor, y_neighbor]
                prob_alien_1[i, j] = p


def init_prob_alien_2(spaceship, bot, apothem):
    cond_outside = condition_outside_detection_square(spaceship, bot, apothem)
    
    cond_pairs_outside = get_pairs_with_replacement(cond_outside)
    
    p = 1 / binom(np.count_nonzero(cond_outside) + 1, 2)
    
    prob_alien_2 = np.where(cond_pairs_outside, p, 0)
    for i in range(0, spaceship.grid_size):
        for j in range(0, spaceship.grid_size):
            prob_alien_2[i, j, i, j] *= 2
    
    return prob_alien_2


def update_prob_alien_2_not_found(prob_alien_2, bot):
    x_bot, y_bot = bot
    
    p_alien_in_bot = np.sum(prob_alien_2[x_bot, y_bot])
    p_alien_not_in_bot = 1 - p_alien_in_bot
    
    prob_alien_2[x_bot, y_bot, :, :] = 0
    prob_alien_2[:, :, x_bot, y_bot] = 0
    prob_alien_2 /= p_alien_not_in_bot


def update_prob_alien_2_detected(prob_alien_2, spaceship, bot, apothem):
    cond_pairs_outside = (
        condition_pairs_with_replacement_outside_detection_square(
            spaceship, bot, apothem
        )
    )
    prob_alien_2[cond_pairs_outside] = 0
    
    sum = np.sum(prob_alien_2) / 2  # prob_alien_2 is symmetric
    prob_alien_2 /= sum


def update_prob_alien_2_not_detected(prob_alien_2, spaceship, bot, apothem):
    cond_pairs_inside = (
        condition_pairs_with_replacement_inside_detection_square(
            spaceship, bot, apothem
        )
    )
    prob_alien_2[cond_pairs_inside] = 0
    
    sum = np.sum(prob_alien_2) / 2  # prob_alien_2 is symmetric
    prob_alien_2 /= sum


def update_prob_alien_2_aliens_move(prob_alien_2, spaceship):
    prob = (
        prob_alien_2
        * spaceship.grid_prob_alien_move_neighbor[:, :, np.newaxis, np.newaxis]
        * spaceship.grid_prob_alien_move_neighbor[np.newaxis, np.newaxis, :, :]
    )
    
    for pair, neighboring_pairs in spaceship.pairs_neighboring_pairs.items():
        jx1, jy1 = pair[0]
        jx2, jy2 = pair[1]
        
        p = 0
        for neighboring_pair in neighboring_pairs:
            kx1, ky1 = neighboring_pair[0]
            kx2, ky2 = neighboring_pair[1]
            if neighboring_pair[0] == neighboring_pair[1]:
                p += prob[kx1, ky1, kx2, ky2] / 2
            else:
                p += prob[kx1, ky1, kx2, ky2]
        
        if pair[0] == pair[1]:
            prob_alien_2[jx1, jy1, jx2, jy2] = p * 2
        else:
            prob_alien_2[jx1, jy1, jx2, jy2] = p
            prob_alien_2[jx2, jy2, jx1, jy1] = p
    
    # normalize
    sum = np.sum(prob_alien_2) / 2  # prob_alien_2 is symmetric
    prob_alien_2 /= sum


def init_prob_crew_1(spaceship, bot):
    x_bot, y_bot = bot
    
    p = 1 / (spaceship.num_open_cells - 1)
    
    prob_crew_1 = np.where(spaceship.grid, p, 0)
    prob_crew_1[x_bot, y_bot] = 0
    
    return prob_crew_1


def update_prob_crew_1_not_found(prob_crew_1, spaceship, bot):
    x_bot, y_bot = bot
    
    p_crew_in_bot = prob_crew_1[x_bot, y_bot]
    
    """
    added if statement to fix error when running Bot 3
        RuntimeWarning: divide by zero encountered in divide
        prob_crew_1 /= p_crew_not_in_bot
    """
    if p_crew_in_bot == 1:
        # init_prob_crew_1() in place
        p = 1 / (spaceship.num_open_cells - 1)
        
        prob_crew_1[spaceship.grid] = p
        prob_crew_1[~spaceship.grid] = 0
        prob_crew_1[x_bot, y_bot] = 0
    else:
        p_crew_not_in_bot = 1 - p_crew_in_bot
        
        prob_crew_1[x_bot, y_bot] = 0
        prob_crew_1 /= p_crew_not_in_bot


def update_prob_crew_1_detected(prob_crew_1, spaceship, bot, alpha):
    for i in range(0, spaceship.grid_size):
        for j in range(0, spaceship.grid_size):
            if spaceship.grid[i][j]:
                p = np.exp(-alpha * (spaceship.distances[bot][(i, j)] - 1))
                prob_crew_1[i, j] *= p
    
    sum = np.sum(prob_crew_1)
    prob_crew_1 /= sum


def update_prob_crew_1_not_detected(prob_crew_1, spaceship, bot, alpha):
    x_bot, y_bot = bot
    
    for i in range(0, spaceship.grid_size):
        for j in range(0, spaceship.grid_size):
            if spaceship.grid[i][j]:
                q = 1 - np.exp(-alpha * (spaceship.distances[bot][(i, j)] - 1))
                prob_crew_1[i, j] *= q
    prob_crew_1[x_bot, y_bot] = 0  # replace negative zero with positive zero
    
    sum = np.sum(prob_crew_1)
    prob_crew_1 /= sum


def init_prob_crew_2(spaceship, bot):
    cond = condition_pairs_without_replacement(spaceship, bot)
    
    p = 1 / (
        binom(spaceship.num_open_cells, 2) - (spaceship.num_open_cells - 1)
    )
    
    prob_crew_2 = np.where(cond, p, 0)
    
    return prob_crew_2


def get_prob_crew_1(prob_crew_2, bot):
    x_bot, y_bot = bot
    
    prob_crew_1 = prob_crew_2[x_bot][y_bot].copy()
    sum = np.sum(prob_crew_1)
    prob_crew_1 /= sum
    
    return prob_crew_1


def update_prob_crew_2_not_found(prob_crew_2, bot):
    x_bot, y_bot = bot
    
    p_crew_in_bot = np.sum(prob_crew_2[x_bot, y_bot])
    p_crew_not_in_bot = 1 - p_crew_in_bot
    
    prob_crew_2[x_bot, y_bot, :, :] = 0
    prob_crew_2[:, :, x_bot, y_bot] = 0
    prob_crew_2 /= p_crew_not_in_bot


def update_prob_crew_2_detected(prob_crew_2, spaceship, bot, alpha):
    grid_size = spaceship.grid_size
    
    prob_j1 = np.zeros((grid_size, grid_size, grid_size, grid_size))
    prob_j2 = np.zeros((grid_size, grid_size, grid_size, grid_size))
    
    for i in range(0, grid_size):
        for j in range(0, grid_size):
            if spaceship.grid[i][j]:
                p = np.exp(-alpha * (spaceship.distances[bot][(i, j)] - 1))
                prob_j1[i, j, :, :] = p
                prob_j2[:, :, i, j] = p
    
    prob_crew_2 *= prob_j1 + prob_j2 - prob_j1 * prob_j2
    
    sum = np.sum(prob_crew_2) / 2  # prob_crew_2 is symmetric
    prob_crew_2 /= sum


def update_prob_crew_2_not_detected(prob_crew_2, spaceship, bot, alpha):
    grid_size = spaceship.grid_size
    x_bot, y_bot = bot
    
    prob_not_j1 = np.zeros((grid_size, grid_size, grid_size, grid_size))
    prob_not_j2 = np.zeros((grid_size, grid_size, grid_size, grid_size))
    
    for i in range(0, grid_size):
        for j in range(0, grid_size):
            if spaceship.grid[i][j]:
                q = 1 - np.exp(-alpha * (spaceship.distances[bot][(i, j)] - 1))
                prob_not_j1[i, j, :, :] = q
                prob_not_j2[:, :, i, j] = q
    
    prob_crew_2 *= prob_not_j1 * prob_not_j2
    
    # replace negative zero with positive zero
    prob_crew_2[x_bot, y_bot, :, :] = 0
    prob_crew_2[:, :, x_bot, y_bot] = 0
    
    sum = np.sum(prob_crew_2) / 2  # prob_crew_2 is symmetric
    prob_crew_2 /= sum


def decide_next_bot_1(spaceship, bot, prob_alien, prob_crew):
    next = get_open_neighbors(bot, spaceship)
    next_prob_crew = {}
    for x, y in next:
        if np.sum(prob_alien[x, y]) == 0:
            next_prob_crew[(x, y)] = np.sum(prob_crew[x, y])
    if not next_prob_crew:
        return bot
    max_prob_crew = max(next_prob_crew.values())
    next = [n for n, p in next_prob_crew.items() if p == max_prob_crew]
    rand_num = randrange(0, len(next))
    return next[rand_num]


def decide_next_bot_2(spaceship, bot, prob_alien, prob_crew):
    next = get_open_neighbors(bot, spaceship)
    next_diff = {}
    for x, y in next:
        next_diff[(x, y)] = np.sum(prob_crew[x, y]) - np.sum(prob_alien[x, y])
    max_diff = max(next_diff.values())
    next = [n for n, p in next_diff.items() if p == max_diff]
    rand_num = randrange(0, len(next))
    return next[rand_num]


def decide_next_bot_5(spaceship, bot, aliens, apothem, prob_alien, prob_crew):
    next = get_open_neighbors(bot, spaceship)
    if detect_alien(bot, aliens, apothem):
        next_prob_alien = {}
        for x, y in next:
            next_prob_alien[(x, y)] = np.sum(prob_alien[x, y])
        min_prob_alien = min(next_prob_alien.values())
        next = [n for n, p in next_prob_alien.items() if p == min_prob_alien]
    else:
        next_prob_crew = {}
        for x, y in next:
            next_prob_crew[(x, y)] = np.sum(prob_crew[x, y])
        max_prob_crew = max(next_prob_crew.values())
        next = [n for n, p in next_prob_crew.items() if p == max_prob_crew]
    rand_num = randrange(0, len(next))
    return next[rand_num]


def decide_next_bot_8(spaceship, bot, aliens, apothem, prob_alien, prob_crew):
    next = get_open_neighbors(bot, spaceship)
    if detect_alien(bot, aliens, apothem):
        next_diff = {}
        for x, y in next:
            next_diff[(x, y)] = (
                np.sum(prob_crew[x, y]) - np.sum(prob_alien[x, y])
            )
        max_diff = max(next_diff.values())
        next = [n for n, p in next_diff.items() if p == max_diff]
    else:
        next_prob_crew = {}
        for x, y in next:
            next_prob_crew[(x, y)] = np.sum(prob_crew[x, y])
        max_prob_crew = max(next_prob_crew.values())
        next = [n for n, p in next_prob_crew.items() if p == max_prob_crew]
    rand_num = randrange(0, len(next))
    return next[rand_num]


def detect_alien(bot, aliens, apothem):
    x_bot, y_bot = bot
    for alien in aliens:
        x_alien, y_alien = alien
        if (abs(x_alien - x_bot) <= apothem and
            abs(y_alien - y_bot) <= apothem):
            return True
    return False


def detect_crew_member(bot, crew_members, alpha, distances):
    for crew_member in crew_members:
        p = np.exp(-alpha * (distances[bot][crew_member] - 1))
        if random() < p:
            return True
    return False


def crew_1_alien_1(
    spaceship, apothem, alpha,
    bot, crew_members, aliens,
    prob_alien_1, prob_crew_1,
    num_saved = 0, num_time_steps = 0, is_success = 0,
    bot_num = 1
):
    while crew_members:
        num_time_steps += 1
        
        if bot_num == 2:
            bot = decide_next_bot_2(spaceship, bot, prob_alien_1, prob_crew_1)
        elif bot_num == 5:
            bot = decide_next_bot_5(
                spaceship, bot, aliens, apothem, prob_alien_1, prob_crew_1
            )
        else:
            bot = decide_next_bot_1(spaceship, bot, prob_alien_1, prob_crew_1)
        
        if bot in aliens:
            break
        else:
            update_prob_alien_1_not_found(prob_alien_1, bot)
        
        if bot in crew_members:
            num_saved += 1
            crew_members.remove(bot)
            if not crew_members:
                is_success = 1
                break
        update_prob_crew_1_not_found(prob_crew_1, spaceship, bot)
        
        if detect_alien(bot, aliens, apothem):
            update_prob_alien_1_detected(
                prob_alien_1, spaceship, bot, apothem
            )
        else:
            update_prob_alien_1_not_detected(
                prob_alien_1, spaceship, bot, apothem
            )
        
        if detect_crew_member(bot, crew_members, alpha, spaceship.distances):
            update_prob_crew_1_detected(prob_crew_1, spaceship, bot, alpha)
        else:
            update_prob_crew_1_not_detected(prob_crew_1, spaceship, bot, alpha)
        
        move_aliens(spaceship, aliens)
        if bot in aliens:
            break
        else:
            update_prob_alien_1_alien_moves(prob_alien_1, spaceship)
    
    return num_saved, num_time_steps, is_success


def crew_2_alien_1_found_1(
    spaceship, apothem, alpha,
    bot, crew_members, aliens,
    prob_alien_1, prob_crew_2,
    num_saved, num_time_steps, is_success,
    bot_num = 4
):
    # reduce prob_crew_2 to prob_crew_1
    prob_crew_1 = get_prob_crew_1(prob_crew_2, bot)
    
    # finish current iteration
    if detect_alien(bot, aliens, apothem):
        update_prob_alien_1_detected(prob_alien_1, spaceship, bot, apothem)
    else:
        update_prob_alien_1_not_detected(prob_alien_1, spaceship, bot, apothem)
    
    if detect_crew_member(bot, crew_members, alpha, spaceship.distances):
        update_prob_crew_1_detected(prob_crew_1, spaceship, bot, alpha)
    else:
        update_prob_crew_1_not_detected(prob_crew_1, spaceship, bot, alpha)
    
    move_aliens(spaceship, aliens)
    if bot in aliens:
        return num_saved, num_time_steps, is_success
    else:
        update_prob_alien_1_alien_moves(prob_alien_1, spaceship)
    
    # start next iteration
    return crew_1_alien_1(
        spaceship, apothem, alpha,
        bot, crew_members, aliens,
        prob_alien_1, prob_crew_1,
        num_saved, num_time_steps, is_success,
        bot_num
    )


def crew_2_alien_1(
    spaceship, apothem, alpha,
    bot, crew_members, aliens,
    prob_alien_1, prob_crew_2,
    num_saved = 0, num_time_steps = 0, is_success = 0,
    bot_num = 4
):
    while crew_members:
        num_time_steps += 1
        
        if bot_num == 5:
            bot = decide_next_bot_5(
                spaceship, bot, aliens, apothem, prob_alien_1, prob_crew_2
            )
        else:
            bot = decide_next_bot_1(spaceship, bot, prob_alien_1, prob_crew_2)
        
        if bot in aliens:
            break
        else:
            update_prob_alien_1_not_found(prob_alien_1, bot)
        
        if bot in crew_members:
            num_saved += 1
            crew_members.remove(bot)
            return crew_2_alien_1_found_1(
                spaceship, apothem, alpha,
                bot, crew_members, aliens,
                prob_alien_1, prob_crew_2,
                num_saved, num_time_steps, is_success,
                bot_num
            )
        else:
            update_prob_crew_2_not_found(prob_crew_2, bot)
        
        if detect_alien(bot, aliens, apothem):
            update_prob_alien_1_detected(
                prob_alien_1, spaceship, bot, apothem
            )
        else:
            update_prob_alien_1_not_detected(
                prob_alien_1, spaceship, bot, apothem
            )
        
        if detect_crew_member(bot, crew_members, alpha, spaceship.distances):
            update_prob_crew_2_detected(prob_crew_2, spaceship, bot, alpha)
        else:
            update_prob_crew_2_not_detected(prob_crew_2, spaceship, bot, alpha)
        
        move_aliens(spaceship, aliens)
        if bot in aliens:
            break
        else:
            update_prob_alien_1_alien_moves(prob_alien_1, spaceship)
    
    return num_saved, num_time_steps, is_success


def crew_1_alien_2(
    spaceship, apothem, alpha,
    bot, crew_members, aliens,
    prob_alien_2, prob_crew_1,
    num_saved = 0, num_time_steps = 0, is_success = 0,
    bot_num = 7
):
    while crew_members:
        num_time_steps += 1
        
        if bot_num == 8:
            bot = decide_next_bot_8(
                spaceship, bot, aliens, apothem, prob_alien_2, prob_crew_1
            )
        else:
            bot = decide_next_bot_1(spaceship, bot, prob_alien_2, prob_crew_1)
        
        if bot in aliens:
            break
        else:
            update_prob_alien_2_not_found(prob_alien_2, bot)
        
        if bot in crew_members:
            num_saved += 1
            crew_members.remove(bot)
            if not crew_members:
                is_success = 1
                break
        update_prob_crew_1_not_found(prob_crew_1, spaceship, bot)
        
        if detect_alien(bot, aliens, apothem):
            update_prob_alien_2_detected(
                prob_alien_2, spaceship, bot, apothem
            )
        else:
            update_prob_alien_2_not_detected(
                prob_alien_2, spaceship, bot, apothem
            )
        
        if detect_crew_member(bot, crew_members, alpha, spaceship.distances):
            update_prob_crew_1_detected(prob_crew_1, spaceship, bot, alpha)
        else:
            update_prob_crew_1_not_detected(prob_crew_1, spaceship, bot, alpha)
        
        move_aliens(spaceship, aliens)
        if bot in aliens:
            break
        else:
            update_prob_alien_2_aliens_move(prob_alien_2, spaceship)
    
    return num_saved, num_time_steps, is_success


def crew_2_alien_2_found_1(
    spaceship, apothem, alpha,
    bot, crew_members, aliens,
    prob_alien_2, prob_crew_2,
    num_saved, num_time_steps, is_success,
    bot_num = 7
):
    # reduce prob_crew_2 to prob_crew_1
    prob_crew_1 = get_prob_crew_1(prob_crew_2, bot)
    
    # finish current iteration
    if detect_alien(bot, aliens, apothem):
        update_prob_alien_2_detected(prob_alien_2, spaceship, bot, apothem)
    else:
        update_prob_alien_2_not_detected(prob_alien_2, spaceship, bot, apothem)
    
    if detect_crew_member(bot, crew_members, alpha, spaceship.distances):
        update_prob_crew_1_detected(prob_crew_1, spaceship, bot, alpha)
    else:
        update_prob_crew_1_not_detected(prob_crew_1, spaceship, bot, alpha)
    
    move_aliens(spaceship, aliens)
    if bot in aliens:
        return num_saved, num_time_steps, is_success
    else:
        update_prob_alien_2_aliens_move(prob_alien_2, spaceship)
    
    # start next iteration
    return crew_1_alien_2(
        spaceship, apothem, alpha,
        bot, crew_members, aliens,
        prob_alien_2, prob_crew_1,
        num_saved, num_time_steps, is_success,
        bot_num
    )


def crew_2_alien_2(
    spaceship, apothem, alpha,
    bot, crew_members, aliens,
    prob_alien_2, prob_crew_2,
    num_saved = 0, num_time_steps = 0, is_success = 0,
    bot_num = 7
):
    while crew_members:
        num_time_steps += 1
        
        if bot_num == 8:
            bot = decide_next_bot_8(
                spaceship, bot, aliens, apothem, prob_alien_2, prob_crew_2
            )
        else:
            bot = decide_next_bot_1(spaceship, bot, prob_alien_2, prob_crew_2)
        
        if bot in aliens:
            break
        else:
            update_prob_alien_2_not_found(prob_alien_2, bot)
        
        if bot in crew_members:
            num_saved += 1
            crew_members.remove(bot)
            return crew_2_alien_2_found_1(
                spaceship, apothem, alpha,
                bot, crew_members, aliens,
                prob_alien_2, prob_crew_2,
                num_saved, num_time_steps, is_success,
                bot_num
            )
        else:
            update_prob_crew_2_not_found(prob_crew_2, bot)
        
        if detect_alien(bot, aliens, apothem):
            update_prob_alien_2_detected(
                prob_alien_2, spaceship, bot, apothem
            )
        else:
            update_prob_alien_2_not_detected(
                prob_alien_2, spaceship, bot, apothem
            )
        
        if detect_crew_member(bot, crew_members, alpha, spaceship.distances):
            update_prob_crew_2_detected(prob_crew_2, spaceship, bot, alpha)
        else:
            update_prob_crew_2_not_detected(prob_crew_2, spaceship, bot, alpha)
        
        move_aliens(spaceship, aliens)
        if bot in aliens:
            break
        else:
            update_prob_alien_2_aliens_move(prob_alien_2, spaceship)
    
    return num_saved, num_time_steps, is_success


def bot_1(spaceship, apothem, alpha):
    bot, crew_members, aliens = place_all(
        spaceship = spaceship,
        num_crew_members = 1,
        num_aliens = 1,
        apothem = apothem
    )
    
    num_saved = 0
    num_time_steps = 0
    is_success = 0
    
    prob_alien_1 = init_prob_alien_1(spaceship, bot, apothem)
    prob_crew_1 = init_prob_crew_1(spaceship, bot)
    
    return crew_1_alien_1(
        spaceship, apothem, alpha,
        bot, crew_members, aliens,
        prob_alien_1, prob_crew_1
    )


def bot_2(spaceship, apothem, alpha):
    bot, crew_members, aliens = place_all(
        spaceship = spaceship,
        num_crew_members = 1,
        num_aliens = 1,
        apothem = apothem
    )
    
    num_saved = 0
    num_time_steps = 0
    is_success = 0
    
    prob_alien_1 = init_prob_alien_1(spaceship, bot, apothem)
    prob_crew_1 = init_prob_crew_1(spaceship, bot)
    
    return crew_1_alien_1(
        spaceship, apothem, alpha,
        bot, crew_members, aliens,
        prob_alien_1, prob_crew_1,
        bot_num = 2
    )


def bot_3(spaceship, apothem, alpha):
    bot, crew_members, aliens = place_all(
        spaceship = spaceship,
        num_crew_members = 2,
        num_aliens = 1,
        apothem = apothem
    )
    
    num_saved = 0
    num_time_steps = 0
    is_success = 0
    
    prob_alien_1 = init_prob_alien_1(spaceship, bot, apothem)
    prob_crew_1 = init_prob_crew_1(spaceship, bot)
    
    return crew_1_alien_1(
        spaceship, apothem, alpha,
        bot, crew_members, aliens,
        prob_alien_1, prob_crew_1
    )


def bot_4(spaceship, apothem, alpha):
    bot, crew_members, aliens = place_all(
        spaceship = spaceship,
        num_crew_members = 2,
        num_aliens = 1,
        apothem = apothem
    )
    
    num_saved = 0
    num_time_steps = 0
    is_success = 0
    
    prob_alien_1 = init_prob_alien_1(spaceship, bot, apothem)
    prob_crew_2 = init_prob_crew_2(spaceship, bot)
    
    return crew_2_alien_1(
        spaceship, apothem, alpha,
        bot, crew_members, aliens,
        prob_alien_1, prob_crew_2
    )


def bot_5(spaceship, apothem, alpha):
    bot, crew_members, aliens = place_all(
        spaceship = spaceship,
        num_crew_members = 2,
        num_aliens = 1,
        apothem = apothem
    )
    
    num_saved = 0
    num_time_steps = 0
    is_success = 0
    
    prob_alien_1 = init_prob_alien_1(spaceship, bot, apothem)
    prob_crew_2 = init_prob_crew_2(spaceship, bot)
    
    return crew_2_alien_1(
        spaceship, apothem, alpha,
        bot, crew_members, aliens,
        prob_alien_1, prob_crew_2,
        bot_num = 5
    )


def bot_6(spaceship, apothem, alpha):
    bot, crew_members, aliens = place_all(
        spaceship = spaceship,
        num_crew_members = 2,
        num_aliens = 2,
        apothem = apothem
    )
    
    num_saved = 0
    num_time_steps = 0
    is_success = 0
    
    prob_alien_1 = init_prob_alien_1(spaceship, bot, apothem)
    prob_crew_1 = init_prob_crew_1(spaceship, bot)
    
    return crew_1_alien_1(
        spaceship, apothem, alpha,
        bot, crew_members, aliens,
        prob_alien_1, prob_crew_1
    )


def bot_7(spaceship, apothem, alpha):
    bot, crew_members, aliens = place_all(
        spaceship = spaceship,
        num_crew_members = 2,
        num_aliens = 2,
        apothem = apothem
    )
    
    num_saved = 0
    num_time_steps = 0
    is_success = 0
    
    prob_alien_2 = init_prob_alien_2(spaceship, bot, apothem)
    prob_crew_2 = init_prob_crew_2(spaceship, bot)
    
    return crew_2_alien_2(
        spaceship, apothem, alpha,
        bot, crew_members, aliens,
        prob_alien_2, prob_crew_2
    )


def bot_8(spaceship, apothem, alpha):
    bot, crew_members, aliens = place_all(
        spaceship = spaceship,
        num_crew_members = 2,
        num_aliens = 2,
        apothem = apothem
    )
    
    num_saved = 0
    num_time_steps = 0
    is_success = 0
    
    prob_alien_2 = init_prob_alien_2(spaceship, bot, apothem)
    prob_crew_2 = init_prob_crew_2(spaceship, bot)
    
    return crew_2_alien_2(
        spaceship, apothem, alpha,
        bot, crew_members, aliens,
        prob_alien_2, prob_crew_2,
        bot_num = 8
    )

Notebook

In [ ]:
def run_experiments(bot_num):
    bot_num_func = {
        1: bot_1,
        2: bot_2,
        3: bot_3,
        4: bot_4,
        5: bot_5,
        6: bot_6,
        7: bot_7,
        8: bot_8
    }
    
    bot_num_num_aliens = {
        1: 1,
        2: 1,
        3: 1,
        4: 1,
        5: 1,
        6: 2,
        7: 2,
        8: 2
    }
    
    grid_size = 20
    
    # apothems = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    apothems = [2]
    alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    num_trials = 100
    
    results_bot = []
    
    for apothem in apothems:
        avg_num_saved_list = []
        avg_num_time_steps_list = []
        success_rate_list = []
        
        for alpha in alphas:
            num_saved_list = []
            num_time_steps_list = []
            is_success_list = []
            
            for _ in range(num_trials):
                spaceship = Spaceship(
                    grid_size = grid_size,
                    num_aliens = bot_num_num_aliens[bot_num]
                )
                
                num_saved, num_time_steps, is_success = (
                    bot_num_func[bot_num](
                        spaceship = spaceship,
                        apothem = apothem,
                        alpha = alpha
                    )
                )
                
                num_saved_list.append(num_saved)
                num_time_steps_list.append(num_time_steps)
                is_success_list.append(is_success)
            
            avg_num_saved = sum(num_saved_list) / num_trials
            avg_num_time_steps = sum(num_time_steps_list) / num_trials
            success_rate = sum(is_success_list) / num_trials
            
            avg_num_saved_list.append(avg_num_saved)
            avg_num_time_steps_list.append(avg_num_time_steps)
            success_rate_list.append(success_rate)
            
            print(f"{apothem}, {alpha}, {avg_num_saved}, {avg_num_time_steps}, {success_rate}")
        
        results_apothem = [
            avg_num_saved_list,
            avg_num_time_steps_list,
            success_rate_list
        ]
        
        results_bot.append(results_apothem)
        
        print("apothem")
        print(apothem)
        print("avg_num_saved_list")
        print(avg_num_saved_list)
        print("avg_num_time_steps_list")
        print(avg_num_time_steps_list)
        print("success_rate_list")
        print(success_rate_list)
    
    print("results_bot")
    print(results_bot)

In [ ]:
run_experiments(1)

In [ ]:
run_experiments(2)

In [ ]:
run_experiments(3)

In [ ]:
run_experiments(4)

In [ ]:
run_experiments(5)

In [ ]:
run_experiments(6)

In [ ]:
run_experiments(7)

In [ ]:
run_experiments(8)

Plots

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
results_bot_1_apothem_all = [
    [
        [0.11, 0.07, 0.08, 0.09, 0.05, 0.08, 0.07, 0.15, 0.07],
        [2179.26, 1975.0, 2094.66, 1739.62, 3055.23, 2011.52, 1924.0, 1956.02, 1700.07],
        [0.11, 0.07, 0.08, 0.09, 0.05, 0.08, 0.07, 0.15, 0.07]
    ],
    [
        [0.68, 0.59, 0.59, 0.59, 0.52, 0.5, 0.55, 0.61, 0.52],
        [476.6, 532.78, 593.01, 612.25, 735.31, 683.15, 667.57, 530.76, 579.97],
        [0.68, 0.59, 0.59, 0.59, 0.52, 0.5, 0.55, 0.61, 0.52]
    ],
    [
        [0.73, 0.58, 0.52, 0.46, 0.54, 0.48, 0.48, 0.5, 0.56],
        [447.22, 549.04, 725.18, 582.89, 673.52, 709.45, 601.32, 624.98, 804.13],
        [0.73, 0.58, 0.52, 0.46, 0.54, 0.48, 0.48, 0.5, 0.56]
    ],
    [
        [0.57, 0.56, 0.59, 0.49, 0.5, 0.46, 0.57, 0.48, 0.43],
        [469.66, 510.21, 658.13, 719.81, 761.29, 698.68, 692.16, 733.27, 577.7],
        [0.57, 0.56, 0.59, 0.49, 0.5, 0.46, 0.57, 0.48, 0.43]
    ],
    [
        [0.56, 0.57, 0.53, 0.58, 0.46, 0.45, 0.47, 0.52, 0.51],
        [523.93, 650.94, 612.49, 710.01, 740.78, 770.63, 888.92, 641.09, 606.28],
        [0.56, 0.57, 0.53, 0.58, 0.46, 0.45, 0.47, 0.52, 0.51]
    ],
    [
        [0.6, 0.49, 0.54, 0.44, 0.46, 0.52, 0.47, 0.49, 0.45],
        [489.81, 520.88, 636.26, 689.13, 753.41, 593.26, 678.89, 690.02, 895.67],
        [0.6, 0.49, 0.54, 0.44, 0.46, 0.52, 0.47, 0.49, 0.45]
    ],
    [
        [0.65, 0.63, 0.42, 0.55, 0.51, 0.51, 0.46, 0.45, 0.42],
        [582.85, 758.13, 680.02, 711.84, 786.05, 736.13, 683.78, 819.13, 712.92],
        [0.65, 0.63, 0.42, 0.55, 0.51, 0.51, 0.46, 0.45, 0.42]
    ],
    [
        [0.65, 0.58, 0.56, 0.51, 0.44, 0.44, 0.49, 0.51, 0.41],
        [493.83, 670.37, 763.6, 839.77, 758.72, 762.02, 892.18, 821.32, 882.19],
        [0.65, 0.58, 0.56, 0.51, 0.44, 0.44, 0.49, 0.51, 0.41]
    ],
    [
        [0.69, 0.52, 0.61, 0.49, 0.44, 0.48, 0.53, 0.47, 0.51],
        [719.33, 841.04, 735.45, 762.78, 883.54, 770.36, 731.52, 783.25, 878.51],
        [0.69, 0.52, 0.61, 0.49, 0.44, 0.48, 0.53, 0.47, 0.51]
    ]
]

In [ ]:
plt.title("Bot 1: Alpha vs. Average number of crew members saved,\nApothem = [1, 9]")
plt.xlabel("Alpha")
plt.ylabel("Average number of crew members saved")
for i in range(0, 9):
    plt.plot(alphas, results_bot_1_apothem_all[i][0], label = i + 1)
plt.legend(title = "Apothem", loc = "lower left", bbox_to_anchor = (1.02, 0), borderaxespad = 0)
plt.show()

In [ ]:
plt.title("Bot 1: Alpha vs. Average number of time steps,\nApothem = [1, 9]")
plt.xlabel("Alpha")
plt.ylabel("Average number of time steps")
for i in range(0, 9):
    plt.plot(alphas, results_bot_1_apothem_all[i][1], label = i + 1)
plt.legend(title = "Apothem", loc = "lower left", bbox_to_anchor = (1.02, 0), borderaxespad = 0)
plt.show()

In [ ]:
plt.title("Bot 1: Alpha vs. Success rate,\nApothem = [1, 9]")
plt.xlabel("Alpha")
plt.ylabel("Success rate")
for i in range(0, 9):
    plt.plot(alphas, results_bot_1_apothem_all[i][2], label = i + 1)
plt.legend(title = "Apothem", loc = "lower left", bbox_to_anchor = (1.02, 0), borderaxespad = 0)
plt.show()

In [ ]:
results_bot_all_apothem_02 = [
    # Bot 1
    [
        [0.68, 0.59, 0.59, 0.59, 0.52, 0.5, 0.55, 0.61, 0.52],
        [476.6, 532.78, 593.01, 612.25, 735.31, 683.15, 667.57, 530.76, 579.97],
        [0.68, 0.59, 0.59, 0.59, 0.52, 0.5, 0.55, 0.61, 0.52]
    ],
    # Bot 2
    [
        [0.82, 0.7, 0.73, 0.66, 0.65, 0.64, 0.66, 0.66, 0.62],
        [544.0, 856.22, 522.76, 778.48, 796.57, 781.73, 889.86, 943.94, 1001.6],
        [0.82, 0.7, 0.73, 0.66, 0.65, 0.64, 0.66, 0.66, 0.62]
    ],
    # Bot 3
    [
        [1.22, 1.23, 1.13, 1.1, 1.09, 0.99, 1.02, 1.19, 1.02],
        [615.39, 857.31, 878.34, 841.68, 824.34, 778.74, 799.89, 883.26, 926.98],
        [0.46, 0.44, 0.41, 0.38, 0.37, 0.35, 0.35, 0.4, 0.3]
    ],
    # Bot 4
    [
        [1.3, 1.13, 1.14, 1.1, 1.2, 1.15, 1.03, 1.08, 1.09],
        [558.82, 727.46, 774.59, 954.5, 792.45, 909.18, 880.58, 946.19, 1014.0],
        [0.48, 0.39, 0.41, 0.4, 0.44, 0.42, 0.37, 0.39, 0.38]
    ],
    # Bot 5
    [
        [1.42, 1.26, 1.37, 1.28, 1.31, 1.17, 1.29, 1.13, 1.21],
        [661.53, 870.74, 1030.56, 855.4, 1150.28, 1297.23, 1161.17, 972.95, 1271.59],
        [0.58, 0.49, 0.55, 0.5, 0.52, 0.43, 0.53, 0.39, 0.42]
    ],
    # Bot 6
    [
        [1.1, 1.0, 1.1, 0.7, 0.7, 0.8, 0.8, 0.8, 0.3],
        [649.5, 712.6, 1005.6, 834.9, 573.1, 597.6, 703.5, 682.0, 543.7],
        [0.4, 0.2, 0.2, 0.2, 0.1, 0.2, 0.2, 0.1, 0.0]
    ],
    # Bot 7
    [
        [1.1, 0.6, 1.0, 0.6, 0.8, 0.7, 0.6, 0.8, 0.6],
        [403.5, 369.4, 468.7, 219.8, 602.2, 767.3, 554.4, 253.6, 483.0],
        [0.4, 0.2, 0.3, 0.2, 0.1, 0.1, 0.1, 0.3, 0.2]
    ],
    # Bot 8
    [
        [1.1, 0.7, 1.1, 0.9, 0.8, 0.4, 0.7, 0.8, 0.9],
        [648.3, 378.7, 709.6, 707.6, 414.6, 361.8, 605.7, 877.3, 648.9],
        [0.4, 0.2, 0.3, 0.2, 0.2, 0.0, 0.1, 0.2, 0.3]
    ]
]

In [ ]:
plt.title("Bot 1 vs. Bot 2: Alpha vs. Average number of crew members saved,\nApothem = 2")
plt.xlabel("Alpha")
plt.ylabel("Average number of crew members saved")
plt.plot(alphas, results_bot_all_apothem_02[0][0], label = "Bot 1")
plt.plot(alphas, results_bot_all_apothem_02[1][0], label = "Bot 2")
plt.legend(loc = "lower left", bbox_to_anchor = (1.02, 0), borderaxespad = 0)
plt.show()

In [ ]:
plt.title("Bot 1 vs. Bot 2: Alpha vs. Average number of time steps,\nApothem = 2")
plt.xlabel("Alpha")
plt.ylabel("Average number of time steps")
plt.plot(alphas, results_bot_all_apothem_02[0][1], label = "Bot 1")
plt.plot(alphas, results_bot_all_apothem_02[1][1], label = "Bot 2")
plt.legend(loc = "lower left", bbox_to_anchor = (1.02, 0), borderaxespad = 0)
plt.show()

In [ ]:
plt.title("Bot 1 vs. Bot 2: Alpha vs. Success rate,\nApothem = 2")
plt.xlabel("Alpha")
plt.ylabel("Success rate")
plt.plot(alphas, results_bot_all_apothem_02[0][2], label = "Bot 1")
plt.plot(alphas, results_bot_all_apothem_02[1][2], label = "Bot 2")
plt.legend(loc = "lower left", bbox_to_anchor = (1.02, 0), borderaxespad = 0)
plt.show()

In [ ]:
plt.title("Bot 3 vs. Bot 4 vs. Bot 5: Alpha vs. Average number of crew members saved,\nApothem = 2")
plt.xlabel("Alpha")
plt.ylabel("Average number of crew members saved")
plt.plot(alphas, results_bot_all_apothem_02[2][0], label = "Bot 3")
plt.plot(alphas, results_bot_all_apothem_02[3][0], label = "Bot 4")
plt.plot(alphas, results_bot_all_apothem_02[4][0], label = "Bot 5")
plt.legend(loc = "lower left", bbox_to_anchor = (1.02, 0), borderaxespad = 0)
plt.show()

In [ ]:
plt.title("Bot 3 vs. Bot 4 vs. Bot 5: Alpha vs. Average number of time steps,\nApothem = 2")
plt.xlabel("Alpha")
plt.ylabel("Average number of time steps")
plt.plot(alphas, results_bot_all_apothem_02[2][1], label = "Bot 3")
plt.plot(alphas, results_bot_all_apothem_02[3][1], label = "Bot 4")
plt.plot(alphas, results_bot_all_apothem_02[4][1], label = "Bot 5")
plt.legend(loc = "lower left", bbox_to_anchor = (1.02, 0), borderaxespad = 0)
plt.show()

In [ ]:
plt.title("Bot 3 vs. Bot 4 vs. Bot 5: Alpha vs. Success rate,\nApothem = 2")
plt.xlabel("Alpha")
plt.ylabel("Success rate")
plt.plot(alphas, results_bot_all_apothem_02[2][2], label = "Bot 3")
plt.plot(alphas, results_bot_all_apothem_02[3][2], label = "Bot 4")
plt.plot(alphas, results_bot_all_apothem_02[4][2], label = "Bot 5")
plt.legend(loc = "lower left", bbox_to_anchor = (1.02, 0), borderaxespad = 0)
plt.show()

In [ ]:
plt.title("Bot 6 vs. Bot 7 vs. Bot 8: Alpha vs. Average number of crew members saved,\nApothem = 2")
plt.xlabel("Alpha")
plt.ylabel("Average number of crew members saved")
plt.plot(alphas, results_bot_all_apothem_02[5][0], label = "Bot 6")
plt.plot(alphas, results_bot_all_apothem_02[6][0], label = "Bot 7")
plt.plot(alphas, results_bot_all_apothem_02[7][0], label = "Bot 8")
plt.legend(loc = "lower left", bbox_to_anchor = (1.02, 0), borderaxespad = 0)
plt.show()

In [ ]:
plt.title("Bot 6 vs. Bot 7 vs. Bot 8: Alpha vs. Average number of time steps,\nApothem = 2")
plt.xlabel("Alpha")
plt.ylabel("Average number of time steps")
plt.plot(alphas, results_bot_all_apothem_02[5][1], label = "Bot 6")
plt.plot(alphas, results_bot_all_apothem_02[6][1], label = "Bot 7")
plt.plot(alphas, results_bot_all_apothem_02[7][1], label = "Bot 8")
plt.legend(loc = "lower left", bbox_to_anchor = (1.02, 0), borderaxespad = 0)
plt.show()

In [ ]:
plt.title("Bot 6 vs. Bot 7 vs. Bot 8: Alpha vs. Success rate,\nApothem = 2")
plt.xlabel("Alpha")
plt.ylabel("Success rate")
plt.plot(alphas, results_bot_all_apothem_02[5][2], label = "Bot 6")
plt.plot(alphas, results_bot_all_apothem_02[6][2], label = "Bot 7")
plt.plot(alphas, results_bot_all_apothem_02[7][2], label = "Bot 8")
plt.legend(loc = "lower left", bbox_to_anchor = (1.02, 0), borderaxespad = 0)
plt.show()